In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/Hindi_train.csv'

df_train = pd.read_csv(file_path)

num_rows = len(df_train)

print(f'The number of rows in the Excel file is: {num_rows}')

The number of rows in the Excel file is: 21225


In [ ]:
df_without_ids = df_train.drop('Id',axis=1)

In [ ]:
df = df_without_ids.drop_duplicates()
# df.describe

In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(df, test_size=0.2)

In [ ]:
!pip install transformers datasets evaluate torch
!pip install accelerate -U
!pip install transformers datasets evaluate torch rouge_score
!pip install rouge bert_score


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset, DatasetDict, load_metric
import numpy as np
import pandas as pd

train_dataset = Dataset.from_pandas(train)
val_dataset = Dataset.from_pandas(val)
dataset = DatasetDict({'train': train_dataset, 'validation': val_dataset})


tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")


def map_to_model_inputs(batch):

    prefix = "summarize: "

    articles = [prefix + article for article in batch['Article']]
    inputs = tokenizer(articles, padding='max_length', truncation=True, max_length=512)
    outputs = tokenizer(batch['Summary'], padding='max_length', truncation=True, max_length=100)

    batch['input_ids'] = inputs.input_ids
    batch['attention_mask'] = inputs.attention_mask
    batch['labels'] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in out] for out in outputs.input_ids
    ]
    return batch


tokenized_dataset = dataset.map(map_to_model_inputs, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    rouge = load_metric('rouge')
    bert_score = load_metric('bertscore')

    rouge_output = rouge.compute(predictions=decoded_preds, references=decoded_labels, rouge_types=["rougeL", "rouge2"])
    bert_score_output = bert_score.compute(predictions=decoded_preds, references=decoded_labels, lang="hi")

    return {
        "rouge2": rouge_output['rouge2'].mid.fmeasure,
        "rougeL": rouge_output['rougeL'].mid.fmeasure,
        "bert_score": bert_score_output['f1'][0]
    }


training_args = Seq2SeqTrainingArguments(
    output_dir='./results_mt5',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.005,
    save_total_limit=1,
    num_train_epochs=5,
    predict_with_generate=True,
    gradient_accumulation_steps = 16
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

model.save_pretrained('/content/drive/MyDrive/Models/mt5_base_finetuned_model')
tokenizer.save_pretrained('/content/drive/MyDrive/Models/mt5_base_finetuned_token')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect t

('/content/drive/MyDrive/Models/mt5_base_finetuned_token/tokenizer_config.json',
 '/content/drive/MyDrive/Models/mt5_base_finetuned_token/special_tokens_map.json',
 '/content/drive/MyDrive/Models/mt5_base_finetuned_token/spiece.model',
 '/content/drive/MyDrive/Models/mt5_base_finetuned_token/added_tokens.json',
 '/content/drive/MyDrive/Models/mt5_base_finetuned_token/tokenizer.json')

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=7a4e1af4b1038577d9c630286de759d37edc2f8ca51270b1f5fdf00a55aab55b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/Models/mt5_base_finetuned_model')
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Models/mt5_base_finetuned_token')

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="/content/drive/MyDrive/Models/mt5_base_finetuned_model", tokenizer="/content/drive/MyDrive/Models/mt5_base_finetuned_token")

text = """मानसून सीजन में हुई भारी बारिश ने कई राज्यों में भारी तबाही मचाई है। हिमाचल प्रदेश में पिछले 40 दिनों में 187 लोगों की जान जा चुकी है, वहीं 34 लोग लापता हैं।
तेलंगाना में एक हफ्ते में 18 लोगों की मौत हो गई। करीब 12,000 लोगों को राहत शिविरों में पहुंचाया गया है।
उधर, शनिवार को जम्मू-कश्मीर के गांदरबल जिले के एक गांव में बादल फट गया। जिससे सात घर, एक मस्जिद और दो स्कूल क्षतिग्रस्त हो गए।
उत्तराखंड के चमोली में रविवार सुबह पहाड़ों से पत्थर गिरने के बाद बद्रीनाथ नेशनल हाईवे को बंद करना पड़ा। हालांकि, 2 घंटे बाद हाईवे ट्रैफिक के लिए खोल दिया गया।
अगले 24 घंटे कैसे रहेंगे…
इन राज्यों में तेज बारिश होगी: हिमाचल, महाराष्ट्र, उत्तराखंड, उत्तर प्रदेश, राजस्थान, मध्यप्रदेश, बिहार, झारखंड, गोवा, छत्तीसगढ़, ओडिशा, पश्चिम बंगाल, असम, अरुणाचल प्रदेश।
बारिश की संभावना नहीं: जम्मू-कश्मीर, पंजाब, हरियाणा, पश्चिमी राजस्थान, गुजरात, कर्नाटक, आंध्र प्रदेश, केरल, तमिलनाडु में बारिश की संभावना नहीं है।
अलग-अलग राज्यों से मानसून की तस्वीरें...
अन्य राज्यों में मौसम का हाल..."""
print(summarizer(text))


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'summary_text': 'महाराष्ट्र में पिछले 40 दिनों में 187 लोगों की जान जा चुकी'}]


In [ ]:
!pip install datasets
!pip install bert_score
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=031201bf378316d2cf347ae5e51c47edc33b7a2a4f7ae87bcb8d36b71fa4a2d6
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from transformers import pipeline
from datasets import load_metric
from bert_score import score as bert_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd


summarizer = pipeline("summarization", model="/content/drive/MyDrive/Models/mt5_base_finetuned_model", tokenizer="/content/drive/MyDrive/Models/mt5_base_finetuned_token")


rouge = load_metric("rouge")


def compute_metrics(generated_summary, reference_summary):

    rouge_scores = rouge.compute(predictions=[generated_summary], references=[reference_summary])

    rouge_scores = {key: value.mid.fmeasure * 100 for key, value in rouge_scores.items()}

    _, _, bertscore_f1 = bert_score([generated_summary], [reference_summary], lang="hi", rescale_with_baseline=False)

    metrics = {k: round(v, 4) for k, v in rouge_scores.items()}
    metrics['bert_score_f1'] = round(bertscore_f1.mean().item() * 100, 4)
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([generated_summary, reference_summary])
    cosine_sim = cosine_similarity(vectors[0:1], vectors[1:])
    metrics['cosine_similarity'] = round(cosine_sim[0][0] * 100, 4)
    return metrics

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`

In [ ]:
for index, row in val.iterrows():

    generated_summaries = summarizer(row["Article"], max_length=80, min_length=30, length_penalty=2.0, num_beams=4)
    generated_summary = generated_summaries[0]['summary_text']

    metrics = compute_metrics(generated_summary, row["Summary"])

    print(f"Article {index}:")
    print(f"Article: {row['Article']}")
    print(f"Generated Summary: {generated_summary}")
    print(f"Reference Summary: {row['Summary']}")
    print(f"Metrics: {metrics}")
    print("\n---\n")

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`

Article 18722:
Article: गुरुग्राम: संयुक्त हिंदू संघर्ष समिति (SHSS) ने घोषणा की है कि वह शहर के खुले इलाकों में जुमे की नमाज का विरोध करने के लिए 5 नवंबर को गुरुग्राम के सेक्टर 12 में गोवर्धन पूजा करेगी। इस अवसर पर संगठन के सदस्य 'अन्नकूट प्रसाद' भी वितरित करेंगे। इसकी घोषणा रविवार को यहां एक संवाददाता सम्मेलन में की गई।सेक्टर-12 वही जगह थी, जहां पुलिस ने 29 अक्टूबर को जुमे की नमाज में खलल डालने के प्रयास में हिंदू संगठनों के 30 सदस्यों को गिरफ्तार किया था।एसएचएसएस हरियाणा प्रदेश अध्यक्ष महावीर भारद्वाज ने पत्रकारों को संबोधित करते हुए कहा कि गुरुग्राम में सार्वजनिक स्थानों पर हो रही नमाज तत्काल बंद की जाए। उन्होंने मुख्यमंत्री से यह भी कहा है कि जिला प्रशासन बिना किसी अधिकारी के हस्ताक्षर वाले शुक्रवार की नमाज के लिए निर्धारित स्थलों की सूची दिखाकर सरकार को गुमराह कर रहा है।भारद्वाज ने हरियाणा सरकार से राज्य में सार्वजनिक स्थानों पर किसी भी धार्मिक प्रथा पर प्रतिबंध लगाने की अधिसूचना जारी करने की मांग की है।
Generated Summary: संयुक्त हिंदू संघर्ष समिति (SHSS) ने घोषणा की है कि वह शह

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Article 5555:
Article: करीब 3 साल पहले अंतरराष्ट्रीय क्रिकेट को अलविदा कह चुके महेंद्र सिंह धोनी की कुल संपत्ति (1,022 करोड़ रु.) 2020 के बाद पहली बार विराट कोहली (998 करोड़ रु.) से ज्यादा हो गई है। टीम इंडिया को टी20 वर्ल्ड चैंपियन, वनडे वर्ल्डकप चैंपियन और 4 बार चेन्नई को आईपीएल चैंपियन बनाने वाले 41 वर्षीय धोनी का क्रेज अब भी वैसा ही है।
यही वजह है कि मौजूद आईपीएल में डिजिटल प्लेटफॉर्म पर अब तक की सबसे ज्यादा व्यूअरशिप (2.4 करोड़) उस वक्त दर्ज की गई, जब धोनी बेंगलुरू के होम ग्राउंड (चिन्नास्वामी स्टेडियम) पर बेंगलुरू के ही खिलाफ क्रीज पर थे। यही नहीं, डिजिटल पर दूसरी सबसे बड़ी व्यूअरशिप (2.2 करोड़) भी तभी आई, जब धोनी राजस्थान के खिलाफ क्रीज पर थे। कोलकाता हो या जयपुर, हर जगह दर्शकदीर्घा में पीली जर्सी पहने धोनी के फैंस सबसे ज्यादा दिखे।
संपत्ति... तीन साल के बाद कोहली को पीछे छोड़ ऐसे आगे बढ़े धोनी
आईपीएल; आज 1000 मैच पूरे करने वाली पहली क्रिकेट लीग बन जाएगी
आईपीएल में कुल 998 मैच हो चुके हैं। रविवार को 2 मैचों के साथ 1000 हो जाएंगे। बिग बैश (552 मैच), पीएसएल (252), सीपीएल (357) जैसी लीग आ

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Article 8168:
Article: पंजाब के मुख्यमंत्री भगवंत मान ने बुधवार को अपने 14 में से 5 मंत्रियों के विभागों में फेरबदल किया। जिन पांच मंत्रियों के विभाग बदले गए हैं, उनमें अमन अरोड़ा, गुरमीत सिंह मीत हेयर, लालजीत सिंह भुल्लर, चेतन सिंह जौड़ामाजरा और अनमोल गगन मान शामिल है। इस फेरबदल में अमन अरोड़ा का रुतबा घटाते हुए उनसे दो बड़े महकमे वापस ले लिए गए।
अमन अरोड़ा ने कुछ दिन पहले ही गैंगस्टरों के हाथों मारे गए पंजाबी सिंगर सिद्धू मूसेवाला को लेकर बयानबाजी की थी। मुख्यमंत्री ने उनसे लोकल बॉडीज और इन्फार्मेशन एंड पब्लिक रिलेशन महकमा वापस लेकर बदले में गवर्नेंस रिफॉर्म्स विभाग दे दिया। यह महकमा अब तक मीत हेयर के पास था।
अमन अरोड़ा के पास अब कुल चार विभागों की जिम्मेदारी रहेगी। अरोड़ा से वापस लिया गया इन्फार्मेशन एंड पब्लिक रिलेशन (PR) महकमा चेतन सिंह जौड़ामाजरा को दे दिया गया वहीं जौड़ामाजरा से फूड प्रोसेसिंग महकमा लेकर लालजीत सिंह भुल्लर को सौंप दिया गया।
मुख्यमंत्री भगवंत मान ने होम, विजिलेंस, इंडस्ट्री, जेल और अर्बन एंड हाउसिंग समेत कुल 10 विभाग अपने पास रखे हैं। CM ने अपने 9 मंत्रियों के महकमों मे

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Article 18916:
Article: चंडीगढ़: पंजाब के मुख्यमंत्री अमरिंदर सिंह ने राज्य में रोजाना कोविड-19 जांच की क्षमता 30,000 तक बढ़ाने के लिए दिशानिर्देश जारी किए हैं। एक बयान में बताया गया कि राज्य में कोविड-19 जांच की मौजूदा क्षमता रोजाना 20,000 है। सरकारी बयान के अनुसार मुख्यमंत्री ने शीर्ष स्वास्थ्य अधिकारियों और चिकित्सा विशेषज्ञों के साथ वीडियो कॉन्फ्रेंस के जरिए बैठक करते हुए राज्य में कोविड-19 स्थिति की समीक्षा की। उन्होंने इसी दौरान जांच क्षमता बढ़ाने का निर्देश दिया। बैठक के दौरान सिंह ने स्वास्थ्य विभाग को निर्देश दिया है कि वे चिकित्सा कॉलेजों और निजी अस्पतालों में वेंटिलेटर की मुफ्त में आपूर्ति करें।पंजाब में पिछले चौबीस घंटे में कोविड-19 से 24 मरीजों की मौत हो गई और संक्रमण के 1,693 नए मामले बुधवार को सामने आए। इसके बाद कोरोना वायरस संक्रमण के कुल मामलों की संख्या राज्य में बढ़कर 36,083 हो गई और महामारी से मरने वालों की संख्या 920 पर पहुंच गई। अधिकारियों ने बुधवार को यह जानकारी दी। चिकित्सकीय बुलेटिन में दी गई जानकारी के अनुसार, लुधियाना में आठ, पटियाला में छह, जालंधर में तीन और 

KeyboardInterrupt: 

In [ ]:
val.shape

(4245, 3)

In [ ]:
import numpy as np

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
bert_scores = []
cosine_similarities = []

test = val[:100]

for index, row in test.iterrows():
    generated_summaries = summarizer(row["Article"], max_length=80, min_length=30, length_penalty=2.0, num_beams=4)
    generated_summary = generated_summaries[0]['summary_text']

    metrics = compute_metrics(generated_summary, row["Summary"])

    rouge1_scores.append(metrics['rouge1'])
    rouge2_scores.append(metrics['rouge2'])
    rougeL_scores.append(metrics['rougeL'])
    bert_scores.append(metrics['bert_score_f1'])
    cosine_similarities.append(metrics['cosine_similarity'])


average_rouge1 = np.mean([score for score in rouge1_scores if score > 0])
average_rouge2 = np.mean([score for score in rouge2_scores if score > 0])
average_rougeL = np.mean([score for score in rougeL_scores if score > 0])
average_bert = np.mean([score for score in bert_scores if score > 0])
average_cosine_similarity = np.mean([score for score in cosine_similarities if score > 0])

print(f"Average ROUGE-1: {average_rouge1:.2f}")
print(f"Average ROUGE-2: {average_rouge2:.2f}")
print(f"Average ROUGE-L: {average_rougeL:.2f}")
print(f"Average BERT: {average_bert:.2f}")
print(f"Average Cosine Similarity: {average_cosine_similarity:.2f}")

Average ROUGE-1: 58.88
Average ROUGE-2: 56.95
Average ROUGE-L: 57.36
Average BERT: 78.95
Average Cosine Similarity: 43.10
